## 深入mnist

In [1]:
import tensorflow as tf 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


## InteractiveSession(),交互环境的简便方法，使用起来总是出错。

### 设置变量（需要优化的参数），占位符（需要传入的数据）

In [3]:
sess=tf.Session()

In [4]:
x = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None, 10])#labels y_
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

## 卷积神经网络理解

### CNN的基本配置---卷积层、Relu层、池化层，全连接。
##### 1，conv,卷积计算层，线性计算
首先对图片数据，找feature，即卷积核。因为样本是二维数据，一般选用3x3,5x5.对于二维数据来说主要设定好三个方向的步进就够{左右方向，上下方向，样本间距？}，对于三维数据多一个feature 步进。如果步进较大可以缩小数据总量。计算出来的新的数据叫：feature map,是已经提取出来的特征，feature map 值越大feature越有意义。
###### 注意：可以选择多个feature（channel?）
#####  2，激活层
方法很多，有logistics函数，tanh和Relu函数。主要作用就是就是对上一步计算结果feature map 进行筛选。非线性激活函数Relu:f(x)=max(0,x)。
##### 3，池化层
对激活层的结果处理。Max Pooling 最大池化、Average Pooling平均池化。选择池化尺寸为2x2，对于max pooling 就是选择这个范围内最大的值当做这个范围的值。
##### 4，全连接。
将筛选后的特征送给分类器，这个时候还需要把四个维度的数据转成二维的>>>样本数X特征数。
###### end，这是对图像的处理，如果对其他样本本来就不是二维数据的情形呢？

### help(tf.truncated_normal)
truncated_normal(shape, mean=0.0, stddev=1.0, dtype=tf.float32, seed=None, name=None)
    
    Outputs random values from a truncated normal distribution.
    
    The generated values follow a normal distribution with specified mean and
    standard deviation, except that values whose magnitude is more than 2 standard
    deviations from the mean are dropped and re-picked.
    

## 1,权重初始化，加入少量的噪声来打破对称性以及避免0梯度

In [6]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

### 2，定义卷积和池化

In [7]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

### conv2d(input, filter, strides, padding, use_cudnn_on_gpu=True, data_format='NHWC', name=None)[num,high,width,channel]
    Computes a 2-D convolution given 4-D `input` and `filter` tensors.    
##### input: A `Tensor`. Must be one of the following types: `half`, `float32`.format,[batch, in_height, in_width, in_channels]
        A 4-D tensor. The dimension order is interpreted according to the value
        of `data_format`, see below for details.
##### filter: A `Tensor`. Must have the same type as `input`.
        `[filter_height, filter_width, in_channels, out_channels]`
##### strides: A list of `ints`.
      1-D tensor of length 4.  The stride of the sliding window for each
        dimension of `input`. The dimension order is determined by the value of
          `data_format`, see below for details.
 ##### padding: A `string` from: `"SAME", "VALID"`.

### max_pool(value, ksize, strides, padding, data_format='NHWC', name=None),

##### ksize: A 1-D int Tensor of 4 elements.  
    The size of the window for each dimension of the input tensor.
#####  strides: A 1-D int Tensor of 4 elements. 
    The stride of the sliding window for each dimension of the input tensor.
      padding: A string, either `'VALID'` or `'SAME'`. The padding algorithm.

#### http://blog.csdn.net/xiaodongxiexie/article/details/74012239
    非常好的博客地址，看了之后才算懂了

In [8]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

    产生一个矩阵作为第一次卷积的权重，fliter格式为[filter_height, filter_width, in_channels, out_channels]`，最后32是我们自己设定的数字，
    就是产生32个feature去卷积，其中的1是因为我们的数据第三个维度高度为1

#### 样本的变化
     我们一共有N个样本，每个样本是748=28x28x1的数据，分别对应图片数据的长宽高
     一般情况下图片都是(RGB)格式，也就是高应该为3。不过mnist数据集的图片只有一个维度。

In [33]:
#把图片转换成需要的格式[NHWC]，这样x_image[0]就是第一个样本了
x_image = tf.reshape(x, [-1,28,28,1])

### 第一次卷积和池化
卷积我们使用的padding=SAME,不会改变大小。池化采用[1,2,2,1],也就是长宽方向各缩减一半其中样本数和通道数不变。图片大小变为14x14

In [11]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

### 第二次卷积和池化

因为上次输出了32个通道，所以这次的输入通道变成32，输出我们用64个。长宽方向各缩减一半，图片大小变为7x7

In [12]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

### Relu激活，教程里没说激活还需要乘以权重啊

##### 池化后样本的形状从[-1,28,28,1]变为[-1,7,7,64]，在把他从4D变成2D,并且第零维度为样本个数。

In [13]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

### dropout(x, keep_prob, noise_shape=None, seed=None, name=None)
    x: A floating point tensor.
    keep_prob: A scalar `Tensor` with the same type as x. The probability
    that each element is kept.保持原样的概率

In [14]:
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

### 全连接

In [15]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [16]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess.run(tf.global_variables_initializer())

### 非常多的变量

In [18]:
for i in range(500):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0},session=sess)
    print("step %d, training accuracy %g"%(i, train_accuracy))
  sess.run(train_step,feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

step 0, training accuracy 0.92
step 100, training accuracy 0.86
step 200, training accuracy 0.98
step 300, training accuracy 0.9
step 400, training accuracy 0.94


In [22]:
print("test accuracy %g"%accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0},session=sess))

test accuracy 0.9502


## 训练模型试一下，改变np.array的数据类型用astype

In [39]:
prex=np.random.rand(784)
prex[prex>0.5]=1.0
prex[prex<=0.5]=0.0
prex=prex.reshape(1,784)
prex=prex.astype('float32')
prex.dtype,prex.shape

(dtype('float32'), (1, 784))

In [24]:
batch[0].dtype,batch[0].shape

(dtype('float32'), (50, 784))

In [25]:
prex.dtype,prex.shape

(dtype('float32'), (1, 784))

##### 怎么两种方法都可以运行了

In [26]:
sess.run(y_conv,feed_dict={x:prex,keep_prob:1.0})

array([[  8.64867587e-03,   5.63948648e-04,   1.34338722e-01,
          8.11128259e-01,   1.33759268e-05,   5.06391469e-03,
          1.65448375e-02,   4.33648791e-04,   2.13822462e-02,
          1.88236905e-03]], dtype=float32)

In [27]:
result=y_conv.eval(feed_dict={x:prex,keep_prob:1.0},session=sess)

In [28]:
result.argmax(),result.max()

(3, 0.81112826)

In [43]:
#第一层卷积
w1=W_conv1.eval(session=sess)
w1.shape

(5, 5, 1, 32)

In [42]:
#第二层卷积
w2=W_conv2.eval(session=sess)
w2.shape

(5, 5, 32, 64)

In [40]:
#激活层
wf1=W_fc1.eval(session=sess)
wf1.shape

(3136, 1024)

In [41]:
#全连接
wf1=W_fc2.eval(session=sess)
wf1.shape,

((1024, 10),)